# MLOps HW1: ML Pipeline с DVC и MLflow

**Задача**: Бинарная классификация медицинских консультаций
- Класс 0: Онлайн консультация
- Класс 1: Требуется офлайн-визит

**GitHub**: https://github.com/kazdoraw/MLOps.git

## 1. Установка зависимостей

In [34]:
!pip install -q pandas numpy scikit-learn mlflow dvc kagglehub joblib matplotlib seaborn pyyaml
!pip install mlflow

## 2. Загрузка датасета ChatDoctor

In [35]:
import kagglehub
import os
import shutil

os.makedirs("data/raw", exist_ok=True)

path = kagglehub.dataset_download("punyaslokaprusty/chatdoctor")
print(f"Загружено в: {path}")

for file in os.listdir(path):
    if os.path.isfile(os.path.join(path, file)):
        shutil.copy(os.path.join(path, file), "data/raw/")
        print(f"Скопирован: {file}")

print("\nJSON файлы готовы")

Загружено в: /Users/kazdoraw/.cache/kagglehub/datasets/punyaslokaprusty/chatdoctor/versions/1
Скопирован: HealthCareMagic-100k.json
Скопирован: iCliniq.json

JSON файлы готовы


## 3. Инициализация Git и DVC

In [36]:
if not os.path.exists('.git'):
    !git init
else:
    print("Git уже существует")

if not os.path.exists('.dvc'):
    !dvc init
    !git add .gitignore .dvc/
    !git commit -m "Init DVC"
else:
    print("DVC уже существует")

Git уже существует
DVC уже существует


## 4. Подготовка данных

`src/prepare.py`:
- Парсит JSON → создает chatdoctor.csv
- Генерирует метки (class_weight будет учтен при обучении)
- Train/test split

In [37]:
!python src/prepare.py

Загрузка существующего CSV: data/raw/chatdoctor.csv
Загружено записей: 231649
Распределение меток: {0: 214502, 1: 17147}

После очистки пропусков: 231649
Распределение меток: {0: 214502, 1: 17147}
Выбрано записей для обучения: 50000

Сохранено:
  - train.csv: 40000 записей
  - test.csv: 10000 записей

Распределение в train: {0: 36985, 1: 3015}
Распределение в test: {0: 9246, 1: 754}


## 5. Обучение модели

`src/train.py`:
- TF-IDF векторизация
- RandomForest с **class_weight='balanced'**
- Логирование в MLflow

In [38]:
!python src/train.py

2025/12/01 00:24:44 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/12/01 00:24:44 INFO mlflow.store.db.utils: Updating database tables
2025-12-01 00:24:44 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-12-01 00:24:44 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025-12-01 00:24:44 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-12-01 00:24:44 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Загрузка обработанных данных...
Train: 40000, Test: 10000
Распределение классов в train: {0: 36985, 1: 3015}
Распределение классов в test: {0: 9246, 1: 754}

Обучение модели...
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.1s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Don

## 6. Просмотр результатов (программно)

In [39]:
import mlflow
import pandas as pd
from IPython.display import display

# Подключаемся к базе данных MLflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")

# Получаем все эксперименты
runs = mlflow.search_runs(
    experiment_names=["chatdoctor_classification"],
    order_by=["start_time DESC"]
)

if runs.empty:
    print("Эксперименты не найдены. Запустите обучение модели.")
else:
    print(f"Найдено экспериментов: {len(runs)}\n")
    
    latest = runs.iloc[0]
    
    print("=" * 60)
    print("РЕЗУЛЬТАТЫ МОДЕЛИ")
    print("=" * 60)
    print(f"\nМодель: {latest.get('params.model_type', 'N/A')}")
    print(f"Дата: {latest['start_time']}")
    
    print("\n" + "-" * 60)
    print("ОБЩИЕ МЕТРИКИ:")
    print("-" * 60)
    print(f"  Accuracy (train):     {latest.get('metrics.accuracy_train', 0):.4f}")
    print(f"  Accuracy (test):      {latest.get('metrics.accuracy_test', 0):.4f}")
    print(f"  Precision (weighted): {latest.get('metrics.precision', 0):.4f}")
    print(f"  Recall (weighted):    {latest.get('metrics.recall', 0):.4f}")
    print(f"  F1-score (weighted):  {latest.get('metrics.f1_score', 0):.4f}")
    
    print("\n" + "-" * 60)
    print("МЕТРИКИ КЛАССА 1 (офлайн-визит):")
    print("-" * 60)
    print(f"  Precision: {latest.get('metrics.precision_class_1', 0):.4f}")
    print(f"  Recall:    {latest.get('metrics.recall_class_1', 0):.4f}")
    print(f"  F1-score:  {latest.get('metrics.f1_score_class_1', 0):.4f}")
    
    print("\n" + "-" * 60)
    print("ПАРАМЕТРЫ:")
    print("-" * 60)
    print(f"  n_estimators:       {latest.get('params.n_estimators', 'N/A')}")
    print(f"  max_depth:          {latest.get('params.max_depth', 'N/A')}")
    print(f"  tfidf_max_features: {latest.get('params.tfidf_max_features', 'N/A')}")
    print("=" * 60)
    
    # Таблица экспериментов
    print("\nВСЕ ЭКСПЕРИМЕНТЫ:")
    cols = ['start_time', 'metrics.accuracy_test', 'metrics.recall_class_1', 'metrics.f1_score_class_1']
    available = [c for c in cols if c in runs.columns]
    if available:
        display(runs[available].head())
    
    print("\nФайлы с результатами:")
    print("   models/confusion_matrix.png")
    print("   models/model.pkl")

Найдено экспериментов: 11

РЕЗУЛЬТАТЫ МОДЕЛИ

Модель: RandomForestClassifier
Дата: 2025-11-30 20:24:45.236000+00:00

------------------------------------------------------------
ОБЩИЕ МЕТРИКИ:
------------------------------------------------------------
  Accuracy (train):     0.9861
  Accuracy (test):      0.9665
  Precision (weighted): 0.9646
  Recall (weighted):    0.9665
  F1-score (weighted):  0.9644

------------------------------------------------------------
МЕТРИКИ КЛАССА 1 (офлайн-визит):
------------------------------------------------------------
  Precision: 0.8631
  Recall:    0.6605
  F1-score:  0.7483

------------------------------------------------------------
ПАРАМЕТРЫ:
------------------------------------------------------------
  n_estimators:       100
  max_depth:          15
  tfidf_max_features: 5000

ВСЕ ЭКСПЕРИМЕНТЫ:


,start_time,metrics.accuracy_test,metrics.recall_class_1,metrics.f1_score_class_1
0,2025-11-30 20:24:45.236000+00:00,0.9665,0.660477,0.74831
1,2025-11-30 20:16:52.281000+00:00,0.9665,0.660477,0.74831
2,2025-11-30 20:14:35.824000+00:00,0.9665,0.660477,0.74831
3,2025-11-30 20:08:31.100000+00:00,0.9665,0.660477,0.74831
4,2025-11-30 20:05:27.972000+00:00,0.9665,0.660477,0.74831



Файлы с результатами:
   models/confusion_matrix.png
   models/model.pkl


## Запуск MLflow UI (Web-интерфейс)

In [40]:
import subprocess
import socket
import time
from IPython.display import display, HTML, IFrame

# Используем порт 5001 вместо 5000 (5000 занят системой)
MLFLOW_PORT = 5001
MLFLOW_URL = f"http://127.0.0.1:{MLFLOW_PORT}"

def is_port_open(port):
    """Проверка, запущен ли сервер на порту"""
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex(('127.0.0.1', port))
    sock.close()
    return result == 0

def check_mlflow_health(port):
    """Проверка, что MLflow UI отвечает"""
    import urllib.request
    try:
        urllib.request.urlopen(f'http://127.0.0.1:{port}/', timeout=2)
        return True
    except:
        return False

if is_port_open(MLFLOW_PORT) and check_mlflow_health(MLFLOW_PORT):
    print(f"MLflow UI запущен на {MLFLOW_URL}\n")
    
    display(HTML(f'''
        <a href="{MLFLOW_URL}" target="_blank" 
           style="display: inline-block; padding: 15px 30px; font-size: 18px; 
                  background-color: #0194E2; color: white; text-decoration: none; 
                  border-radius: 5px; font-weight: bold; margin: 10px 0;">
            Открыть MLflow UI
        </a>
    '''))
    
    print("\nMLflow UI (встроенный):")
    display(IFrame(src=MLFLOW_URL, width=1000, height=600))
    
else:
    print(f"MLflow UI НЕ запущен на порту {MLFLOW_PORT}\n")
    print("ЗАПУСК:\n")
    print("В терминале выполните:")
    print("   cd /Users/Shared/ml/DEPLOY/HW1")
    print("   ./start_mlflow.sh")
    print(f"\nЗатем откройте: {MLFLOW_URL}")

MLflow UI запущен на http://127.0.0.1:5001




MLflow UI (встроенный):
